## Section 1: Environment Setup

First, let's import the necessary libraries and configure our Azure OpenAI connection.

In [ ]:
# Import required libraries
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from datetime import datetime, timedelta
import os
import time
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

# Initialize Azure OpenAI LLM
llm = AzureChatOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_version="2024-10-21",
        api_key=AZURE_OPENAI_KEY,
        azure_deployment=AZURE_OPENAI_DEPLOYMENT 
    )

print("✓ Environment configured successfully!")

## Section 2: Sample Customer Data

Let's create a mock database with customer accounts for testing.

In [ ]:
# Mock customer database
CUSTOMER_DATA = {
    "CUST001": {
        "name": "Alice Johnson",
        "accounts": {
            "CHK001": {"type": "checking", "balance": 5000.00},
            "SAV001": {"type": "savings", "balance": 15000.00}
        },
        "transactions": [
            {"date": "2025-12-15", "type": "deposit", "account": "CHK001", "amount": 2000.00, "description": "Salary"},
            {"date": "2025-12-14", "type": "withdrawal", "account": "CHK001", "amount": 150.00, "description": "ATM Withdrawal"},
            {"date": "2025-12-13", "type": "purchase", "account": "CHK001", "amount": 85.50, "description": "Grocery Store"},
            {"date": "2025-12-12", "type": "transfer", "account": "SAV001", "amount": 500.00, "description": "Transfer from CHK001"},
            {"date": "2025-12-10", "type": "interest", "account": "SAV001", "amount": 25.00, "description": "Monthly Interest"}
        ]
    },
    "CUST002": {
        "name": "Bob Smith",
        "accounts": {
            "CHK002": {"type": "checking", "balance": 3200.00},
            "SAV002": {"type": "savings", "balance": 8500.00}
        },
        "transactions": [
            {"date": "2025-12-15", "type": "deposit", "account": "CHK002", "amount": 1500.00, "description": "Paycheck"},
            {"date": "2025-12-14", "type": "purchase", "account": "CHK002", "amount": 200.00, "description": "Online Shopping"}
        ]
    }
}

print(f"✓ Loaded {len(CUSTOMER_DATA)} customers with mock data")

## Section 3: Create a tool - Account Balance Inquiry

Create a tool that retrieves account balance information.

In [ ]:
@tool
def get_account_balance(customer_id: str, account_id: str = None) -> str:
    """Get account balance for a customer. 
    
    Args:
        customer_id: Customer ID (e.g., CUST001)
        account_id: Optional specific account ID (e.g., CHK001). If not provided, returns all accounts.
    
    Returns:
        Account balance information as a formatted string
    """
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    
    if account_id:
        if account_id not in customer["accounts"]:
            return f"Error: Account {account_id} not found for customer {customer_id}."
        account = customer["accounts"][account_id]
        return f"{customer['name']}'s {account['type']} account ({account_id}): ${account['balance']:,.2f}"
    else:
        result = f"Account balances for {customer['name']} ({customer_id}):\n"
        for acc_id, acc_data in customer["accounts"].items():
            result += f"  - {acc_data['type'].capitalize()} ({acc_id}): ${acc_data['balance']:,.2f}\n"
        return result

# Test the tool
print(get_account_balance.invoke({"customer_id": "CUST001"}))

## Section 4: Define the Banking Agent

Now let's create the agent with all tools and configure it with a banking-specific prompt.

In [ ]:
# Collect all tools
banking_tools = [get_account_balance]

# Define agent prompt template
banking_prompt = """You are a helpful banking assistant. You have access to tools to help customers with their banking needs.

Always be polite, professional, and ensure you're helping the correct customer given the customer ID. 
"""


# Create the agent
agent = create_react_agent(
    model=llm,
    tools=banking_tools,
    prompt=banking_prompt
)


print("✓ Banking agent created successfully!")

## Section 5: Simple Query Tests

Test the agent with simple, single-tool queries.

In [ ]:
# Test 1: Balance inquiry
print("=" * 60)
print("TEST 1: Balance Inquiry")
print("=" * 60)
response = agent.invoke({"messages": "What is the balance for customer id CUST001?"})
all_messages = response['messages']
final_messages = all_messages[1:]

print("\nFinal Answer:", final_messages[-1].content)

## Section 6: Error Cases and Validation

Test how the agent handles errors and invalid inputs.

In [ ]:
# Test 5: Invalid customer ID
print("=" * 60)
print("TEST 5: Invalid Customer ID")
print("=" * 60)
response = agent.invoke({"messages": "What's the balance for customer id CUST999?"})
all_messages = response['messages']
final_messages = all_messages[1:]

print("\nFinal Answer:", final_messages[-1].content)

In [ ]:
# Test 6: Invalid ask 
print("=" * 60)
print("TEST 6: Insufficient Funds")
print("=" * 60)
response = agent.invoke({
    "messages": "Transfer $100 from CHK001 to SAV001 for customer CUST001"
})
all_messages = response['messages']
final_messages = all_messages[1:]

print("\nFinal Answer:", final_messages[-1].content)

In [ ]:
# Test 7: Invalid account
print("=" * 60)
print("TEST 7: Invalid Account")
print("=" * 60)
response = agent.invoke({
    "messages": "What's the balance of account XYZ999 for customer CUST001?"
})
all_messages = response['messages']
final_messages = all_messages[1:]

print("\nFinal Answer:", final_messages[-1].content)

## Section 8: Performance Evaluation

Measure agent performance metrics including response time and tool usage.

In [ ]:
# Performance test
def measure_performance(query: str, test_name: str):
    """Measure agent performance for a given query."""
    print(f"\n{'=' * 60}")
    print(f"Performance Test: {test_name}")
    print(f"{'=' * 60}")
    print(f"Query: {query}\n")
    
    start_time = time.time()
    response = agent.invoke({"messages": query})
    end_time = time.time()
    all_messages = response['messages']
    final_messages = all_messages[1:]

    elapsed_time = end_time - start_time
    
    print(f"\n--- Performance Metrics ---")
    print(f"Response Time: {elapsed_time:.2f} seconds")
    print(f"Final Answer Length: {len(final_messages[-1].content)} characters")
    print(f"\nFinal Answer: {final_messages[-1].content}")
    
    return {
        "test_name": test_name,
        "response_time": elapsed_time,
        "answer_length": len(final_messages[-1].content)
    }

# Run performance tests
results = []
results.append(measure_performance(
    "What are all the account balances for customer CUST001?",
    "Simple Balance Query"
))

# Summary
print(f"\n\n{'=' * 60}")
print("PERFORMANCE SUMMARY")
print(f"{'=' * 60}")
for result in results:
    print(f"{result['test_name']}: {result['response_time']:.2f}s")
print(f"\nAverage Response Time: {sum(r['response_time'] for r in results) / len(results):.2f}s")

## Workshop Complete! 🎉

You've successfully built a banking agent that can:
- ✓ Check account balances
- ✓ Handle errors gracefully

### Next Steps:
1. **Extend Tools**: Add more banking tools.
2. **Multi-Agent**: Convert to multi-agent system with specialized agents.